# 0. Install and Import Dependencies

In [1]:
import mediapipe as mp
import cv2
import numpy as np
import csv
import warnings
import tensorflow as tf
import pandas as pd
import pickle
warnings.filterwarnings('ignore')

In [2]:
mp_drawing = mp.solutions.drawing_utils #Drawing helpers
mp_pose = mp.solutions.pose

# 1. Load Model

In [3]:
# path to the model
dir_model = "../model/xyz/"

# you can try different model
# dir_model = "../Model/xyz_hand/"
# dir_model = "../Model/xy/"
# dir_model = "../Model/xy_hand/"

loaded_model = tf.saved_model.load(dir_model)
inferer = loaded_model.signatures["serving_default"]

# 2. Setup OpenCV and MediaPipe to Predict

In [7]:
dir_video_test = ('../../data/all videos/perfect.mp4')
cap = cv2.VideoCapture(dir_video_test)
current_stage = ''

#initiate holistic model
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    # stream the video
    while cap.isOpened():
        ret, frame = cap.read()
        
        # recolor feed
        # image = cv2.flip(frame, 1)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # make detection
        results = pose.process(image)
        
        # recolor image back to BGR for rendering
        image.flags.writeable = False
        image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

        # try to predict
        try:
            # model xyz
            row = np.array([[res.x, res.y, res.z] for res in results.pose_world_landmarks.landmark]).flatten()
            X = pd.DataFrame([row])
            ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xyz_hand
            # row = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            #ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            #ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[]]).flatten()
            # X = pd.DataFrame([row])
            #ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            #ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            # model xy_hand
            # row = np.array([[res.x, res.y] for res in results.pose_landmarks.landmark[13:22]]).flatten()
            # X = pd.DataFrame([row])
            #ROM_class = np.argmax(inferer(inputs=X)['output_0'].numpy())
            #ROM_prob = np.max(inferer(inputs=X)['output_0'].numpy())

            print(ROM_class,ROM_prob)

            if ROM_class == 0 and ROM_prob >= 0.9 :
                current_stage = 'NOT'
            elif ROM_class == 1 and ROM_prob >= 0.9 :
                current_stage = 'YES!'

            # get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

            #Display class
            cv2.putText(image, 'ROM', (95, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, current_stage, (95, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

            #Display Prob
            cv2.putText(image, 'PROB', (15, 12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(ROM_prob, 2)), (15, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    
            
        except Exception as e:
            print(e)
            
        # Stream vid resultq
        cv2.imshow("Raw Cam Feed", image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1 0.8221642
1 0.80673164
1 0.736024
1 0.6796354
1 0.67559755
1 0.78352314
1 0.8470342
1 0.8765267
1 0.8977651
1 0.9066269
1 0.92582333
1 0.9331355
1 0.9373749
1 0.93319064
1 0.93600106
1 0.91036785
1 0.9246083
1 0.9434904
1 0.9233445
1 0.9079145
1 0.8710962
0 0.61834425
0 0.934307
0 0.98836493
0 0.9908563
0 0.99733174
0 0.9976051
0 0.9978031
0 0.9983298
0 0.99864143
0 0.9987815
0 0.9960035
0 0.90961796
0 0.5924551
1 0.6371238
1 0.93741983
1 0.9895915
1 0.9950062
1 0.9995592
1 0.9997982
1 0.99982786
1 0.9998919
1 0.99994016
1 0.99995345
1 0.99998647
1 0.99998933
1 0.99997914
1 0.99999464
1 0.99999726
1 0.9999993
1 0.99999774
1 0.999999
1 0.999997
1 0.9999939
1 0.99999774
1 0.99999726
1 0.9999905
1 0.9999861
1 0.99995965
1 0.99988705
1 0.99978405
1 0.9997464
1 0.9988225
1 0.99783707
1 0.99649036
1 0.9877957
1 0.91175526
1 0.9089769
0 0.5628353
0 0.77690715
0 0.777325
0 0.863777
0 0.92427003
0 0.9392302
0 0.93652856
0 0.9765346
0 0.96529967
0 0.9766571
0 0.9545796
0 0.9472737
0 0.6159848
